# Try Reproducing Everything to Stock

In [62]:
import sys
import os
sys.path.append(os.path.abspath("D:\\Python\\commodity_futures_price"))

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

# %load_ext autoreload
# %autoreload 2
%reload_ext autoreload
from model.savvy_sh import SavvySh

import yfinance as yf

from model.factor import Factor

### Download Necessary Data From yfinance (if haven't downloaded)

In [85]:
tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA", "JPM", "BAC", "WFC",
    "GS", "V", "MA", "UNH", "PFE", "JNJ", "MRK", "WMT", "COST", "PG",
    "KO", "PEP", "XOM", "CVX", "COP", "CAT", "DE", "NKE", "DIS", "NFLX",
    "RUN", "FSLR", "PLUG", "CHWY", "DKNG", "ROKU", "U", "HOOD", "UPST", "AFRM"
]
for ticker in tickers:
    # download if not found in ../../data/yfinance/
    if not os.path.exists(f"../../data/yfinance/{ticker}.csv"):
        data = yf.Ticker(ticker)
        data = data.history(period="15Y", repair=True)
        data.index = data.index.date
        data.to_csv(f"../../data/yfinance/{ticker}.csv")
    data = pd.read_csv(f"../../data/yfinance/{ticker}.csv", index_col=0, parse_dates=True)
    # check if there is any row with Repair = True
    data[data['Repaired?'] == True]

    # adding factors
    factor = Factor(data, ticker=ticker)
    factor.add_momentum()
    factor.add_skewness()
    factor.add_volatility()
    factor.add_liquidity()
    # factor.add_value()
    factor.add_market_cap(method='log')
    
    # save the data
    data.to_csv(f'../../data/processed_stock/cleaned_{ticker}.csv')